# Use-case 4: Social Media Analysis

---
---

## Data preparation for social media analysis

In [1]:
# Import the necessary libraries
import json
import pandas as pd

# Read in the json file
with open('data/banijay_op1data_twitter_raw.json', 'rb') as f:
    twitter_json = json.load(f)

# Convert the json file to a pandas dataframe
df_op1_twitter = pd.json_normalize(twitter_json)

df_op1_twitter.head()

,referenced_tweets,possibly_sensitive,lang,text,edit_history_tweet_ids,author_id,created_at,id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,author.id,author.public_metrics.followers_count,author.public_metrics.following_count,author.public_metrics.tweet_count,author.public_metrics.listed_count,author.name,author.username
0,"[{'type': 'replied_to', 'id': '160025955556878...",False,nl,Lees hier het gesprek terug: #Op1 \n\nhttps://...,[1600440065410453509],17022916,2022-12-07T10:40:20.000Z,1600440065410453509,0,8,0,0,17022916,337078,1432,38840,2362,Op1,op1npo
1,"[{'type': 'replied_to', 'id': '160025483213200...",False,nl,Lees hier het gesprek terug: #Op1 \n\nhttps://...,[1600440000520372225],17022916,2022-12-07T10:40:05.000Z,1600440000520372225,0,0,0,0,17022916,337078,1432,38840,2362,Op1,op1npo
2,"[{'type': 'replied_to', 'id': '160025268126773...",False,nl,@mariettehamer Lees hier het gesprek terug: #O...,[1600431542878314497],17022916,2022-12-07T10:06:29.000Z,1600431542878314497,0,1,0,0,17022916,337078,1432,38840,2362,Op1,op1npo
3,"[{'type': 'replied_to', 'id': '160025066981831...",False,nl,@fleurjongepier Lees hier het gesprek terug: #...,[1600431528479178754],17022916,2022-12-07T10:06:25.000Z,1600431528479178754,0,0,2,0,17022916,337078,1432,38840,2362,Op1,op1npo
4,"[{'type': 'replied_to', 'id': '160024856209630...",False,nl,@robdewijk Lees hier het gesprek terug: #Op1 \...,[1600426335301767169],17022916,2022-12-07T09:45:47.000Z,1600426335301767169,1,0,1,0,17022916,337078,1432,38840,2362,Op1,op1npo


In [2]:
df_content_ratings = pd.read_csv('data/banijay_op1data_content_ratings.csv', compression = 'zip') #Reading the merged file of content and ratings.

---

## Pre-processing the data

### Only retain the tweets that are not a reference to another tweet.
### You can do this by filtering out the tweets that have a referenced_tweets column with a NaN value.

In [3]:
df_op1_twitter = df_op1_twitter[df_op1_twitter['referenced_tweets'].isna()]

In [4]:
# Dropping columns that are not needed.
df_op1_twitter.drop('referenced_tweets', inplace=True, axis=1)
df_op1_twitter.drop('text', inplace=True, axis=1)
df_op1_twitter.drop('possibly_sensitive', inplace=True, axis=1)
df_op1_twitter.drop('author_id', inplace=True, axis=1)
df_op1_twitter.drop('edit_history_tweet_ids', inplace=True, axis=1)
df_op1_twitter.drop('id', inplace=True, axis=1)
df_op1_twitter.drop('author.name', inplace=True, axis=1)
df_op1_twitter.drop('author.username', inplace=True, axis=1)
df_op1_twitter.drop('author.id', inplace=True, axis=1)
df_op1_twitter.drop('author.public_metrics.tweet_count', inplace=True, axis=1)
df_op1_twitter.drop('author.public_metrics.listed_count', inplace=True, axis=1)
df_op1_twitter.drop('lang', inplace=True, axis=1)
df_op1_twitter.drop('author.public_metrics.following_count',
                    inplace=True, axis=1)

In [5]:
df_op1_twitter.head(3) #Checking if the drop code provided above worked.

,created_at,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,author.public_metrics.followers_count
5,2022-12-07T07:01:00.000Z,0,27,3,1,337078
6,2022-12-07T06:41:00.000Z,0,0,2,0,337078
7,2022-12-07T06:21:00.000Z,5,6,4,0,337078


## Merging the datasets

### Convert the created_at column to a datetime object.

In [6]:
df_op1_twitter.dtypes #Checking the data types of the columns.

created_at                               object
public_metrics.retweet_count              int64
public_metrics.reply_count                int64
public_metrics.like_count                 int64
public_metrics.quote_count                int64
author.public_metrics.followers_count     int64
dtype: object

In [7]:
#Rename created_at to Datum in order to merge with the content_ratings merged file, since it is called Datum in that file.
df_op1_twitter.rename(columns={'created_at': 'Datum'}, inplace=True)

In [8]:
df_op1_twitter['Datum'] = df_op1_twitter['Datum'].apply(
    lambda row: row.split("T")[0]) #Splitting the date and time on the T and keeping only the date.

In [9]:
df_op1_twitter.head(3) #Checking if the code above worked.

,Datum,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,author.public_metrics.followers_count
5,2022-12-07,0,27,3,1,337078
6,2022-12-07,0,0,2,0,337078
7,2022-12-07,5,6,4,0,337078


In [10]:
df_op1_twitter['Datum'] = pd.to_datetime(df_op1_twitter['Datum']) #Converting Datum to a datetime object.

In [11]:
df_op1_twitter.sort_values(by='Datum', inplace=True) #Sorting the DataFrame by Datum.

In [12]:
#Grouping the DataFrame by Datum and summing the retweet_count, reply_count, like_count and quote_count columns and taking the average of the followers_count column.
df_op1_twitter = df_op1_twitter.groupby('Datum').agg({'public_metrics.retweet_count': 'sum', 'public_metrics.reply_count': 'sum',
                                                      'public_metrics.like_count': 'sum', 'public_metrics.quote_count': 'sum', 'author.public_metrics.followers_count': 'mean'})

In [13]:
df_op1_twitter.head(3)

,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,author.public_metrics.followers_count
Datum,,,,,
2020-01-06,64,109,249,45,337078.0
2020-01-07,135,112,420,62,337078.0
2020-01-08,111,142,436,76,337078.0


In [14]:
df_op1_twitter = df_op1_twitter.reset_index() #Resetting the index, so that Datum is a column again.

In [15]:
df_op1_twitter.head(3) #Checking if the code above worked.

,Datum,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,author.public_metrics.followers_count
0,2020-01-06,64,109,249,45,337078.0
1,2020-01-07,135,112,420,62,337078.0
2,2020-01-08,111,142,436,76,337078.0


In [16]:
df_content_ratings.info() #Checking the data types of the columns.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858449 entries, 0 to 858448
Data columns (total 25 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       858449 non-null  int64  
 1   Datum            858449 non-null  object 
 2   Time             858449 non-null  object 
 3   Program          858449 non-null  object 
 4   Station          858449 non-null  object 
 5   Target Group     858449 non-null  object 
 6   Broadcast Type   858449 non-null  object 
 7   Ratings Type     858449 non-null  object 
 8   Kdh%             858449 non-null  float64
 9   Kdh000           858449 non-null  float64
 10  Zadl%            858449 non-null  float64
 11  date_time        858449 non-null  object 
 12  id               858449 non-null  object 
 13  date             858449 non-null  object 
 14  hosts            858449 non-null  object 
 15  length           858449 non-null  object 
 16  start            858449 non-null  obje

In [17]:
df_content_ratings['Datum'] = pd.to_datetime(df_content_ratings['Datum']) #Converting the column Datum of df_content_ratings to a datetime object so I will be able to merge the 2 DataFrames together.

In [18]:
df_content_ratings.info() #Checking if the code above worked and if the column Datum is now a datetime object.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858449 entries, 0 to 858448
Data columns (total 25 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Unnamed: 0       858449 non-null  int64         
 1   Datum            858449 non-null  datetime64[ns]
 2   Time             858449 non-null  object        
 3   Program          858449 non-null  object        
 4   Station          858449 non-null  object        
 5   Target Group     858449 non-null  object        
 6   Broadcast Type   858449 non-null  object        
 7   Ratings Type     858449 non-null  object        
 8   Kdh%             858449 non-null  float64       
 9   Kdh000           858449 non-null  float64       
 10  Zadl%            858449 non-null  float64       
 11  date_time        858449 non-null  object        
 12  id               858449 non-null  object        
 13  date             858449 non-null  object        
 14  hosts            858

In [19]:
df_merged_twitter = pd.merge(df_content_ratings, df_op1_twitter, on='Datum', how="left") #Merging the 2 DataFrames together.

In [20]:
df_merged_twitter.head() #Checking if the merge worked.

,Unnamed: 0,Datum,Time,Program,Station,Target Group,Broadcast Type,Ratings Type,Kdh%,Kdh000,...,keywords,date_time_start,date_time_end,show_id,fragment,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,author.public_metrics.followers_count
0,0,2020-01-06,22:18:00,op1,npo1,tot6plus,live/prerecorded uitzendingen,uitzenddag,9.676969,1546.863437,...,"['anderhalve', 'racismedemonstratie', 'hoofdst...",2020-01-06 22:17:59,2020-01-06 22:39:40,OP1__________-WON02123548,1_segment,64,109,249,45,337078.0
1,1,2020-01-06,22:18:00,op1,npo1,tot6plus,live/prerecorded uitzendingen,uitgesteld,1.484044,237.224411,...,"['anderhalve', 'racismedemonstratie', 'hoofdst...",2020-01-06 22:17:59,2020-01-06 22:39:40,OP1__________-WON02123548,1_segment,64,109,249,45,337078.0
2,2,2020-01-06,22:18:00,op1,npo1,tot6plus,live/prerecorded uitzendingen,totaal,10.299407,1646.360210,...,"['anderhalve', 'racismedemonstratie', 'hoofdst...",2020-01-06 22:17:59,2020-01-06 22:39:40,OP1__________-WON02123548,1_segment,64,109,249,45,337078.0
3,3,2020-01-06,22:18:00,op1,npo1,boodschapper_20_49,live/prerecorded uitzendingen,uitzenddag,3.217735,117.833443,...,"['anderhalve', 'racismedemonstratie', 'hoofdst...",2020-01-06 22:17:59,2020-01-06 22:39:40,OP1__________-WON02123548,1_segment,64,109,249,45,337078.0
4,4,2020-01-06,22:18:00,op1,npo1,boodschapper_20_49,live/prerecorded uitzendingen,uitgesteld,1.146627,41.989484,...,"['anderhalve', 'racismedemonstratie', 'hoofdst...",2020-01-06 22:17:59,2020-01-06 22:39:40,OP1__________-WON02123548,1_segment,64,109,249,45,337078.0


---

In [21]:
df_merged_twitter.to_csv('data/banijay_op1data_content_ratings_twitter.csv', compression='zip') #Saving my progress and compressing the data to be able to upload it to GitHub.